In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU, Input, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from skimage.transform import resize
from skimage.io import imread

# --- 1. CONFIGURACIÓN ---
DIRECTORIO_DATASET = os.path.abspath("./dataset") 
ANCHO = 64  
ALTO = 64
CANALES = 3 

print(f"Buscando en: {DIRECTORIO_DATASET}")

if not os.path.exists(DIRECTORIO_DATASET):
    print(f"❌ ERROR: No encuentro la carpeta '{DIRECTORIO_DATASET}'. Asegúrate de estar en el directorio correcto.")
    exit()

# --- 2. CARGA DE DATOS ---
# Detectar clases
clases_validas = [d for d in os.listdir(DIRECTORIO_DATASET) if os.path.isdir(os.path.join(DIRECTORIO_DATASET, d))]
clases_validas.sort()
mapa_clases = {nombre: i for i, nombre in enumerate(clases_validas)}

print(f"Clases detectadas ({len(clases_validas)}): {clases_validas}")

images = []
labels = []

print("Cargando imágenes...")
for nombre_clase in clases_validas:
    ruta_clase = os.path.join(DIRECTORIO_DATASET, nombre_clase)
    archivos = os.listdir(ruta_clase)
    
    count = 0
    for archivo in archivos:
        if archivo.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp')):
            ruta_imagen = os.path.join(ruta_clase, archivo)
            try:
                img = imread(ruta_imagen)
                
                # Quitar transparencia (Alpha) si existe
                if len(img.shape) == 3 and img.shape[2] == 4:
                     img = img[:, :, :3]
                
                # Redimensionar
                img_resized = resize(img, (ALTO, ANCHO), anti_aliasing=True, preserve_range=True)
                
                # Asegurar 3 canales RGB
                if len(img_resized.shape) == 3 and img_resized.shape[2] == 3:
                    images.append(img_resized)
                    labels.append(mapa_clases[nombre_clase]) 
                    count += 1
            except Exception as e:
                pass
    
    print(f"    {nombre_clase}: {count} imágenes")

# --- 3. PREPROCESAMIENTO ---
if len(images) == 0:
    print("❌ Error: No se cargaron imágenes. Revisa tu dataset.")
    exit()

print("Procesando datos...")
X = np.array(images, dtype=np.float32) / 255.0 
y = np.array(labels)
y_one_hot = to_categorical(y, num_classes=len(clases_validas))

print(f"Datos listos: X={X.shape}, y={y_one_hot.shape}")

# Separar Train/Test
train_X, test_X, train_Y, test_Y = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# --- 4. DATA AUGMENTATION (MEJORA DE ROBUSTEZ) ---
# Crea variaciones de las imágenes en tiempo real para evitar overfitting
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

# --- 5. MODELO MEJORADO ---
model = Sequential()
model.add(Input(shape=(ALTO, ANCHO, CANALES)))

# Bloque 1
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization()) # Normaliza para aprender más rápido
model.add(LeakyReLU(negative_slope=0.1))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25)) # Apaga neuronas al azar para evitar memorización

# Bloque 2
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU(negative_slope=0.1))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# Bloque 3 (Más profundo para características complejas)
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU(negative_slope=0.1))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))

# Clasificador (Dense)
model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(LeakyReLU(negative_slope=0.1))
model.add(Dropout(0.5))

model.add(Dense(len(clases_validas), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# --- 6. CALLBACKS (SALVAVIDAS) ---
callbacks_list = [
    # Parar si no mejora la validación en 10 épocas
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    # Guardar solo el mejor modelo
    ModelCheckpoint('mejor_modelo.h5', monitor='val_loss', save_best_only=True, verbose=1),
    # Bajar la velocidad de aprendizaje si se estanca
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
]

# --- 7. ENTRENAMIENTO ---
print("Iniciando entrenamiento robusto...")
# Usamos datagen.flow para el set de entrenamiento
history = model.fit(
    datagen.flow(train_X, train_Y, batch_size=32),
    epochs=100, # Ponemos un tope alto, EarlyStopping lo detendrá antes si es necesario
    validation_data=(test_X, test_Y),
    callbacks=callbacks_list
)

# --- 8. GUARDAR Y GRAFICAR ---
model.save("modelo_final_robusto.h5")
np.save('clases.npy', clases_validas)
print("Archivos guardados: 'modelo_final_robusto.h5' y 'clases.npy'")

# Gráficas de rendimiento
plt.figure(figsize=(12, 5))

# Gráfica de Precisión (Accuracy)
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión del Modelo')
plt.xlabel('Épocas')
plt.ylabel('Accuracy')
plt.legend()

# Gráfica de Pérdida (Loss)
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida (Loss)')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()

plt.show()

Configurando mejoras de robustez...
Iniciando entrenamiento robusto...
Epoch 1/100
 63/529 ━━━━━━━━━━━━━━━━━━━━ 50s 109ms/step - accuracy: 0.4358 - loss: 1.5006

KeyboardInterrupt: 